### Финальный проект karpov.courses: вариант 2
#### Задание 3

В базе данных вашей компании присутствуют следующие таблицы

    city
        city_id – id города, где проходит акция
        client_city_id – id города, где находится покупатель
        city – город
        client_city – город, где находится покупатель
    client
        client_id – id покупателя
        client_city_id – id города, где находится покупатель
        birth_date – дата рождения покупателя
        registration – дата регистрации покупателя
    promotion
        promotion_id – id акции
        category_id – id категории
        promotion_name – акция
        category_name – категория
        partner_id – id партнёра
        partner_name – партнёр
    purchase
        purchase_id – id покупки
        partner_id – id партнёра
        client_id – id покупателя
        city_id – id города
        promotion_id – id акции
        category_id – id категории
        purchase_date – дата покупки
        price – цена за единицу товара
        quantity – число проданных единиц
        status – статус покупки

Вам нужно написать запрос, чтобы получить такую таблицу

    purchase_date – дата покупки
    purchase_id – id покупки
    client_id – id покупателя
    client_age – возраст покупателя
    client_registration_age – как долго человек пользуется вашими магазинами
    client_category – содержит new или old, если это первая или последующая покупка соответственно
    promotion_name – акция
    category_name – категория
    partner_name – партнёр
    client_city – город, где находится покупатель
    city – город
    revenue – сумма выручки
    quantity – число проданных единиц

При этом в таблице должны присутствовать только значения, где в поле status стоит значение 1, и только те, где purchase_date находится в диапазоне от 01.05.2020 до 01.08.2020

In [1]:
# У меня дома развернуты свои виртуалки с ClickHouse и Redash,
# поэтому мне было удобнее решать задачу в рамках своей инфраструктуры.
# Думаю, это не принципиально, ведь я планирую не использовать каких-то специфичных для ClickHouse SQL-конструкций.
from simplech import ClickHouse
# https://rock.st/docs/reference/simplech/
# очень простая библиотека для работы с ClickHouse

In [17]:
# подключаемся БД и проверяем что все ОК
ch = ClickHouse(host='192.168.15.115', password='***')
print(ch.select('SHOW DATABASES'))

_temporary_and_external_tables
default
project_2_3
system
test



In [3]:
# создаем отдельную БД для таблиц проекта
ch.run('CREATE DATABASE IF NOT EXISTS project_2_3')

In [4]:
# создаем сами таблицы
# city
query = """\
CREATE TABLE IF NOT EXISTS project_2_3.city (
    city_id UInt64,
    client_city_id UInt64,
    city String,
    client_city String
    ) ENGINE=Log
"""
ch.run(query)
# движок Log, а не MergeTree() просто для простоты,
# ведь мне таблицы нужны будут не для реальной работы,
# а просто для экспериментов с итоговым запросом.

In [5]:
# client
query = """\
CREATE TABLE IF NOT EXISTS project_2_3.client (
    client_id UInt64,
    client_city_id UInt64,
    birth_date Date,
    registration Date
    ) ENGINE=Log
"""
ch.run(query)

In [6]:
# promotion
query = """\
CREATE TABLE IF NOT EXISTS project_2_3.promotion (
    promotion_id UInt64,
    category_id UInt64,
    promotion_name String,
    category_name String,
    partner_id UInt64,
    partner_name String
    ) ENGINE=Log
"""
ch.run(query)

In [18]:
# purchase
query = """\
CREATE TABLE IF NOT EXISTS project_2_3.purchase (
    purchase_id UInt64,
    partner_id UInt64,
    client_id UInt64,
    city_id UInt64,
    promotion_id UInt64,
    category_id UInt64,
    purchase_date Date,
    price Float64,
    quantity Float64,
    status UInt8
    ) ENGINE=Log
"""
ch.run(query)

Ниже будет очень прямолинейный подход к заполнению таблиц данными.
Правильно было бы использовать бибилиотеку Faker https://faker.readthedocs.io/en/master/, чтобы сделать небольшие списки городов, клиентов и т.д.
И затем случаным образом выбирать элементы списков для заполнения таблиц. Айдишниками бы выступали порядковые номера элементов списков. Но время уже так сильно поджимало, что пришлось делать по простому.

In [8]:
# запишем даныне в city
ch.push('project_2_3.city', {'city_id': 1, 'client_city_id': 1, 'city': 'Anapa', 'client_city': 'Anapa'})
ch.push('project_2_3.city', {'city_id': 2, 'client_city_id': 2, 'city': 'Moscow', 'client_city': 'Moscow'})
ch.push('project_2_3.city', {'city_id': 3, 'client_city_id': 3, 'city': 'Krasnodar', 'client_city': 'Krasnodar'})
ch.flush_all()

In [9]:
# запишем даныне в client
ch.push('project_2_3.client', {
    'client_id': 1, 'client_city_id': 1, 'birth_date': '1982-01-29', 'registration': '2020-10-01'})
ch.push('project_2_3.client', {
    'client_id': 2, 'client_city_id': 1, 'birth_date': '1983-02-28', 'registration': '2020-10-02'})
ch.push('project_2_3.client', {
    'client_id': 3, 'client_city_id': 2, 'birth_date': '1973-05-01', 'registration': '2020-10-01'})
ch.push('project_2_3.client', {
    'client_id': 4, 'client_city_id': 2, 'birth_date': '1980-06-09', 'registration': '2020-10-05'})
ch.push('project_2_3.client', {
    'client_id': 5, 'client_city_id': 3, 'birth_date': '1991-01-01', 'registration': '2020-10-06'})
ch.flush_all()

In [10]:
# запишем даныне в promotion
ch.push('project_2_3.promotion', {
    'promotion_id': 1, 'category_id': 1, 'promotion_name': 'Milka Chocolate', 'category_name': 'food',
    'partner_id': 1, 'partner_name': 'Mars'})
ch.push('project_2_3.promotion', {
    'promotion_id': 2, 'category_id': 2, 'promotion_name': 'Football events', 'category_name': 'sport',
    'partner_id': 2, 'partner_name': 'Addidas'})
ch.push('project_2_3.promotion', {
    'promotion_id': 3, 'category_id': 2, 'promotion_name': 'Running events', 'category_name': 'sport', 
    'partner_id': 2, 'partner_name': 'Addidas'})
ch.flush_all()

In [19]:
# запишем даныне в purchase
ch.push('project_2_3.purchase', {
    'purchase_id': 1, 'partner_id': 1, 'client_id': 1, 'city_id': 2, 'promotion_id': 1, 'category_id': 1,
    'purchase_date': '2020-02-01', 'price': 325, 'quantity': 1, 'status': 1})

ch.push('project_2_3.purchase', {
    'purchase_id': 2, 'partner_id': 1, 'client_id': 2, 'city_id': 2, 'promotion_id': 1, 'category_id': 1,
    'purchase_date': '2020-05-02', 'price': 500, 'quantity': 2, 'status': 1})

ch.push('project_2_3.purchase', {
    'purchase_id': 3, 'partner_id': 1, 'client_id': 2, 'city_id': 1, 'promotion_id': 1, 'category_id': 1,
    'purchase_date': '2020-05-03', 'price': 500, 'quantity': 1, 'status': 1})

ch.push('project_2_3.purchase', {
    'purchase_id': 4, 'partner_id': 2, 'client_id': 3, 'city_id': 2, 'promotion_id': 2, 'category_id': 2,
    'purchase_date': '2020-06-10', 'price': 890.15, 'quantity': 3, 'status': 1})

ch.push('project_2_3.purchase', {
    'purchase_id': 5, 'partner_id': 2, 'client_id': 4, 'city_id': 2, 'promotion_id': 2, 'category_id': 2,
    'purchase_date': '2020-06-15', 'price': 750.05, 'quantity': 2, 'status': 0})

ch.push('project_2_3.purchase', {
    'purchase_id': 6, 'partner_id': 2, 'client_id': 3, 'city_id': 2, 'promotion_id': 3, 'category_id': 2,
    'purchase_date': '2020-08-10', 'price': 900, 'quantity': 1, 'status': 1})

ch.push('project_2_3.purchase', {
    'purchase_id': 7, 'partner_id': 2, 'client_id': 3, 'city_id': 2, 'promotion_id': 3, 'category_id': 2,
    'purchase_date': '2020-07-20', 'price': 800, 'quantity': 2, 'status': 1})

ch.push('project_2_3.purchase', {
    'purchase_id': 8, 'partner_id': 2, 'client_id': 5, 'city_id': 3, 'promotion_id': 3, 'category_id': 2,
    'purchase_date': '2020-07-20', 'price': 850, 'quantity': 2, 'status': 1})

ch.flush_all()

In [22]:
# Итоговый запрос для полечения результата
query = """\
SELECT 
    purchase_date,
    purchase_id,
    purchase.client_id AS client_id,
    DATEDIFF(YY, birth_date, now()) AS client_age_years,
    DATEDIFF(DD, registration, now()) AS client_registration_age_days,
    if(purchase_date = x.first_purchase, 'new', 'old') AS client_category,
    promotion_name,
    category_name,
    partner_name,
    c2.client_city AS client_city2,
    c1.city AS city1,
    price * quantity AS revenue,
    quantity
FROM project_2_3.purchase
LEFT JOIN project_2_3.client
ON purchase.client_id = client.client_id
LEFT JOIN project_2_3.promotion AS pr
ON purchase.promotion_id = pr.promotion_id
LEFT JOIN project_2_3.city AS c1
ON purchase.city_id = c1.city_id
LEFT JOIN project_2_3.city AS c2
ON client.client_city_id  = c2.client_city_id 
LEFT JOIN (
    SELECT client_id,
      MIN(purchase_date) AS first_purchase
    FROM project_2_3.purchase
    GROUP BY client_id
) AS x
ON purchase.client_id = x.client_id
WHERE status=1 AND purchase_date between '2020/05/01' AND '2020/08/01'
"""
print(ch.select(query))
# Пришлось изменить накоторые элиасы:
# у client_age и sclient_registration_age добавил к названию размерность,
# потому что в задании не уточнялось - возраст в годах/днях.
# Изменил client_city на client_city2, потому что иначе, видимо, значение выводилось из первого джойна c1

2020-05-02	2	2	37	24	new	Milka Chocolate	food	Mars	Anapa	Moscow	1000	2
2020-05-03	3	2	37	24	old	Milka Chocolate	food	Mars	Anapa	Anapa	500	1
2020-06-10	4	3	47	25	new	Football events	sport	Addidas	Moscow	Moscow	2670.45	3
2020-07-20	7	3	47	25	old	Running events	sport	Addidas	Moscow	Moscow	1600	2
2020-07-20	8	5	29	20	new	Running events	sport	Addidas	Krasnodar	Krasnodar	1700	2

